#### Import Julia packages

In [9]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [10]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Makie
# using Interpolations

ArgumentError: ArgumentError: Package Makie not found in current path, maybe you meant `import/using .Makie`.
- Otherwise, run `import Pkg; Pkg.add("Makie")` to install the Makie package.

#### Import Custom functions

In [11]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

#### Read JSON File

In [12]:
width, height, Tini_json, Tfin_json, windows, doors, heating_elem = get_room("Boundary.json")

(5, 3, 283.15, 293.15, Any[(0.75125, 2.32125, "Left"), (0.73125, 2.32125, "Right")], Any[(0.235, 0.655, "Top")], [[2.005, 3.435, 0.10125, 0.20125], [1.765, 3.215, 2.80125, 2.92125]])

In [13]:
# price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.11781, 0.11106, 0.10592, 0.10341, 0.10836, 0.10655, 0.16935, 0.43496, 0.39644], Any[1.38209, 1.3029, 1.2426, 1.21315, 1.27123, 1.24999, 1.98673, 5.10273, 4.65084], Any[282.15, 282.34999999999997, 282.25, 282.34999999999997, 282.15, 281.45, 280.65, 280.84999999999997, 280.75], Any[10, 11, 12, 13, 14, 15, 16, 17, 18])

#### Boundary conditions

In [14]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [15]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition)
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

UnconstrainedFESpace()

#### Time parameters

In [16]:
t0 = 0.0    # Start time
tF = 28800   # End time
Δt = 10

10

#### Room parameters

In [17]:
h_wall(x) = 0.22*0.02           # [W/m^2K]
h_window(x) = 1.2*0.02
h_door(x) = 1.0*0.02 #mindre?

ρ(x)=1.225
c(x)=1020.0
k(x)=5#3.5


h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 


Tout = get_temp_function(T_out, tF)

Tini(x)= Tini_json
TFin(x)= Tfin_json

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(TFin, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 625
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 11429712145970673821

#### Controll parameter

In [18]:
q_pos = heating_elements(heating_elem)

Q(x,t)=0
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [19]:
L2norm(u)=√(Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1)))
L2skp(u)=Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1))
get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)


Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=400*(∑(∫(q_pos)*dΩ))
println(b)
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

100.79999999999991


Proj (generic function with 2 methods)

#### Test Gradient DEs

In [20]:
price = t -> 0

γ = 1.225*1020
Q(x,t)= 0
Qt(t)=x->Q(x,t)

(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=2,
armijoparas=(ρ=0.5, α_0=10, α_min=1e-3, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.1835186925152044e6, k = 1
α = 10, new_cost = 719756.5350543942, L2fgrad = 4210.837480599833, interm = 0.0
entered for loop, E=719756.5350543942, k = 2
α = 10, new_cost = 461496.02260891744, L2fgrad = 3279.476388035381, interm = 0.0


(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (28710.0, SingleFieldFEFunction()), (28720.0, SingleFieldFEFunction()), (28730.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28750.0, SingleFieldFEFunction()), (28760.0, SingleFieldFEFunction()), (28770.0, SingleFieldFEFunction()), (28780.0, SingleFieldFEFunction()), (28790.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [21]:
# price = t -> 0
# c1_zero = (plot_costs2(costs))
# CairoMakie.save("price_zero_convergence.png", c1_zero)


# Q_int = get_control_int(qs)
# T_int = get_temperature_int(Ts)
# c2_zero = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
# CairoMakie.save("price_zero_result.png", c2_zero)

In [ ]:
price = t -> 0.4 + χ(t,-1,3000)*2 + χ(t,10000,15000)*2 + χ(t,23000,28800+1)*2
γ = 1.225*1020*2

(Ts2,qs2,Ws2,costs2) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=100,
armijoparas=(ρ=0.5, α_0=10.0, α_min=1e-4, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=2.367037385030409e6, k = 1
α = 10.0, new_cost = 5.179552050437054e6, L2fgrad = 7569.8674673147225, interm = 0.0
α = 5.0, new_cost = 1.6318496462540384e6, L2fgrad = 7569.8674673147225, interm = 0.0
entered for loop, E=1.6318496462540384e6, k = 2
α = 10.0, new_cost = 2.367037385030409e6, L2fgrad = 6064.269017705042, interm = 0.0
α = 5.0, new_cost = 1.088431823080291e6, L2fgrad = 6064.269017705042, interm = 0.0
entered for loop, E=1.088431823080291e6, k = 3
α = 10.0, new_cost = 3.5642173010544274e6, L2fgrad = 4594.456755659124, interm = 0.0
α = 5.0, new_cost = 787060.0259705059, L2fgrad = 4594.456755659124, interm = 0.0
entered for loop, E=787060.0259705059, k = 4
α = 10.0, new_cost = 1.7108386610212363e6, L2fgrad = 3692.47341651157, interm = 0.0
α = 5.0, new_cost = 548107.2661926748, L2fgrad = 3692.47341651157, interm = 0.0
entered for loop, E=548107.2661926748, k = 5
α = 10.0, new_cost = 1.5807341513693691e6, L2fgrad = 2580.955616975944, interm 

In [ ]:
price = t -> 0.4 + χ(t,-1,3000)*2 + χ(t,10000,15000)*2 + χ(t,23000,28800+1)*2

# c1_step = (plot_costs(costs2))
c1_step = (plot_costs2(costs2))

CairoMakie.save("price_step_convergence.png", c1_step)


Q_int = get_control_int(qs2)
T_int = get_temperature_int(Ts2)
c2_step = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_step_result.png", c2_step)



In [ ]:
price = get_price_function(price_NOK, tF, 1)
γ = 1.225*1020*3

(Ts3,qs3,Ws3,costs3) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=100,
armijoparas=(ρ=0.5, α_0=10, α_min=1e-4, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

In [ ]:
price = get_price_function(price_NOK, tF, 1)

# c1_real = (plot_costs(costs3))
c1_real = (plot_costs2(costs3))

CairoMakie.save("price_real_convergence.png", c1_real)

Q_int = get_control_int(qs3)
T_int = get_temperature_int(Ts3)
c2_real = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_real_result.png", c2_real)


In [ ]:
price = t -> 1
γ = 1.225*1020*2

(Ts4,qs4,Ws4,costs4) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=100,
armijoparas=(ρ=0.5, α_0=20, α_min=1e-5, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

In [ ]:
price = t -> 1

# c1_const = (plot_costs(costs4))
c1_const = (plot_costs2(costs4))

CairoMakie.save("price_const_convergence.png", c1_const)

Q_int = get_control_int(qs4)
T_int = get_temperature_int(Ts4)
c2_const = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_const_result.png", c2_const)


In [ ]:
# # Opprett en mappe for midlertidige filer hvis den ikke eksisterer
# if !isdir("tmp")
#     mkdir("tmp")
# end

# # Initialtilstanden og resten av tidsløsningene
# # uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# # uh  = solution[2:end]  # De resterende løsningene


# uh = Ts3
# # Lagre resultater i Paraview-format (VTK)
# createpvd("resultsOutisde") do pvd
#     # Første løsning (t=0)
#     # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

#     # Lagrer løsninger for hver tidssteg
#     count = 0
#     for (tn, uhn) in uh
#         if count%6 == 0
#             pvd[tn] = createvtk(Ω, "tmp/results_$count.vtu", cellfields=["u" => uhn])
#         end
#         count+=1
#     end
# end


In [ ]:
get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)


function real_price(qs,price)
    price1 = 0
    for (t,QQ) in qs
        price1 += Δt*price(t)*∑(∫(QQ)*dΩ)/3600/1000
    end
    return price1
end

price = get_price_function(price_NOK, tF, 1)
println(real_price(qs,price))
# println(get_temperature_int(Ts-Tfin))


function result_plot(;T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    time = LinRange(t0, tF, length(T_int))

    fig = CairoMakie.Figure(size=(600,900))

    ax = CairoMakie.Axis(fig[1,1], title="Temperature profile", xlabel="Time (hours)", ylabel="Temperature (°C)")
    lines!(ax, time, T_int, color=:green) #label="Temp"

    ax=CairoMakie.Axis(fig[2,1], title="Optimal Heating Schedule", xlabel="Time (hours)", ylabel="Heating Output (W)")  
    lines!(ax, time, Q_int, color=:red)

    ax=CairoMakie.Axis(fig[3,1], title="Energy price over time", xlabel="Time (hours)", ylabel="Price (NOK)")
    lines!(ax, hour_lst, price_NOK, color=:blue)

    ax=CairoMakie.Axis(fig[4,1], title="Outside Temperature over time", xlabel="Time (hours)", ylabel="Temperature (°C)")
    lines!(ax, hour_lst, T_out, color=:orange)

    return fig
end

function result_plot2(; T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    # time = LinRange(t0, tF, length(T_int))
    time = LinRange(t0, tF, length(T_int)) ./ 3600  # convert from seconds to hours
    time2 = LinRange(t0, tF, length(T_int))

    
    time_price = [price(t) for t in time2]

    fig = CairoMakie.Figure(size=(600, 900), figure_padding=(0, 5, 0, 5))

    # Formatter for temperature axis (Kelvin to Celsius)
    temp_formatter = v -> ["$(round(vi - 273.15, digits=1))" for vi in v]

    # Temperature profile
    ax = CairoMakie.Axis(
        fig[1, 1],
        title = "Temperature profile",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, time, T_int, color = :green)

    # Heating output
    ax = CairoMakie.Axis(
        fig[2, 1],
        title = "Optimal Heating Schedule",
        xlabel = "Time (hours)",
        ylabel = "Heating Output (W)"
    )
    lines!(ax, time, Q_int, color = :red)

    # Energy price
    ax = CairoMakie.Axis(
        fig[3, 1],
        title = "Energy price over time",
        xlabel = "Time (hours)",
        ylabel = "Price (NOK)"
    )
    lines!(ax, time, time_price, color = :blue)

    # Outside temperature
    ax = CairoMakie.Axis(
        fig[4, 1],
        title = "Outside Temperature over time",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, hour_lst .-10, T_out, color = :orange)

    return fig
end

function plot_costs(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost"#,
        # yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end

function plot_costs2(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost",
        yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end
